raw2text

In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("datapath")
print(dataset)

In [2]:
dataset[1]

{'token_count': 365,
 'language': 'en',
 'text': 'COOLIEF radiofrequency is a new non-invasive, outpatient treatment option for those suffering from knee pain, hip pain, SI joint, and chronic back pain. This advanced procedure uses radiofrequency energy to safely target the sensory nerves causing pain. COOLIEF circulates water through the device that serves as a heat sink while heating nerves to create a treatment area that is larger than conventional radiofrequency ablation. This combination targets the pain-causing nerves without excessive heating or damage to surrounding tissues. I have performed this procedure with great success for many patients. It is safe for patients who have undergone joint replacement because this procedure blocks nerve pain before the nerves enter the joint space.\nMedical marijuana has been in the news a lot lately. Since it became legal under Florida state law for certain medical conditions, many people have asked me about whether it is right for them or t

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("path/to/qwen72b")

In [6]:
# This code is used to check the length added after adding chat_template.
inputs = []
prompts = [""]
messages_template = [
    {"role": "system", "content": "You are a professional doctor and can determine whether the following conversations involve the medical field."},
    {"role": "user", "content": "{prompt}"}
]

def create_messages(prompt):
    return [
        msg.copy() if msg["role"] == "system" 
        else {"role": "user", "content": prompt}
        for msg in messages_template
    ]

for prompt in prompts:
    messages = create_messages(prompt)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    inputs.append(text)
len(tokenizer.encode(inputs[0]))

30

In [4]:
p = """
You need to generate two questions based on the given text content. These questions can be open-ended, detail-oriented, or related to a broader discussion of the content, but avoid relying on specific case details from the text. Follow these requirements:
Requirements:
1. Make sure the questions are closely related to the main points or themes mentioned in the text.
2. Ensure the two questions are as diverse as possible, avoiding homogeneity.
3. Ensure the questions include all the information needed for the answers. If necessary, add introductory information to the questions.
4. Avoid repetitive or overly simplistic questions, ensuring diversity and depth.
5. The questions must be self-contained and should not require the provided text as background to be understood.
Please rewrite the following text into related questions, and strictly follow the format below for output:
{
    "question1": "Generated first question",
    "question2": "Generated second question"
}
Here is the sample text:
"""
p

'\nYou need to generate two questions based on the given text content. These questions can be open-ended, detail-oriented, or related to a broader discussion of the content, but avoid relying on specific case details from the text. Follow these requirements:\nRequirements:\n1. Make sure the questions are closely related to the main points or themes mentioned in the text.\n2. Ensure the two questions are as diverse as possible, avoiding homogeneity.\n3. Ensure the questions include all the information needed for the answers. If necessary, add introductory information to the questions.\n4. Avoid repetitive or overly simplistic questions, ensuring diversity and depth.\n5. The questions must be self-contained and should not require the provided text as background to be understood.\nPlease rewrite the following text into related questions, and strictly follow the format below for output:\n{\n    "question1": "Generated first question",\n    "question2": "Generated second question"\n}\nHere 

In [4]:
def raw2text(example):
    # Use a list to store string fragments and then join them all at once
    text_parts = ['You need to generate two questions based on the given text content. These questions can be open-ended, detail-oriented, or related to a broader discussion of the content, but avoid relying on specific case details from the text. Follow these requirements:\nRequirements:\n1. Make sure the questions are closely related to the main points or themes mentioned in the text.\n2. Ensure the two questions are as diverse as possible, avoiding homogeneity.\n3. Ensure the questions include all the information needed for the answers. If necessary, add introductory information to the questions.\n4. Avoid repetitive or overly simplistic questions, ensuring diversity and depth.\n5. The questions must be self-contained and should not require the provided text as background to be understood.\nPlease rewrite the following text into related questions, and strictly follow the format below for output:\n{\n    "question1": "Generated first question",\n    "question2": "Generated second question"\n}\nHere is the sample text:\n']
    
    text_parts.append(example["text"])
    
    text = "".join(text_parts)
    # Truncate the text so that the total length does not exceed 8192 after adding it to chat_template
    if example["token_count"] > 7965:
        text = tokenizer.decode(tokenizer.encode(text, truncation=True, max_length=7965))

    text += "\nOutput: \n"
    example["new_text"] = text
    return example

In [5]:
refined_dataset = dataset.map(raw2text,remove_columns=dataset.column_names)
print(refined_dataset)

Map (num_proc=128): 100%|██████████| 210000/210000 [02:12<00:00, 1590.72 examples/s]


Dataset({
    features: ['new_text'],
    num_rows: 210000
})


In [6]:
refined_dataset[0]

{'new_text': 'You need to generate two questions based on the given text content. These questions can be open-ended, detail-oriented, or related to a broader discussion of the content, but avoid relying on specific case details from the text. Follow these requirements:\nRequirements:\n1. Make sure the questions are closely related to the main points or themes mentioned in the text.\n2. Ensure the two questions are as diverse as possible, avoiding homogeneity.\n3. Ensure the questions include all the information needed for the answers. If necessary, add introductory information to the questions.\n4. Avoid repetitive or overly simplistic questions, ensuring diversity and depth.\n5. The questions must be self-contained and should not require the provided text as background to be understood.\nPlease rewrite the following text into related questions, and strictly follow the format below for output:\n{\n    "question1": "Generated first question",\n    "question2": "Generated second question

In [7]:
refined_dataset = refined_dataset.rename_column("new_text", "text")
refined_dataset

Dataset({
    features: ['text'],
    num_rows: 210000
})

text2prompt

In [8]:
messages_template = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "{prompt}"}
]

def create_messages(prompt):
    return [
        msg.copy() if msg["role"] == "system" 
        else {"role": "user", "content": prompt}
        for msg in messages_template
    ]

In [9]:
def create_prompt(example):
    message = create_messages(example["text"])
    text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True
    )
    example["prompt"] = text
    return example

In [10]:
refined_dataset = refined_dataset.map(create_prompt, remove_columns=["text"], num_proc=128)
refined_dataset

Map (num_proc=128): 100%|██████████| 210000/210000 [00:19<00:00, 10969.79 examples/s]


Dataset({
    features: ['prompt'],
    num_rows: 210000
})

In [11]:
refined_dataset[0]

{'prompt': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nYou need to generate two questions based on the given text content. These questions can be open-ended, detail-oriented, or related to a broader discussion of the content, but avoid relying on specific case details from the text. Follow these requirements:\nRequirements:\n1. Make sure the questions are closely related to the main points or themes mentioned in the text.\n2. Ensure the two questions are as diverse as possible, avoiding homogeneity.\n3. Ensure the questions include all the information needed for the answers. If necessary, add introductory information to the questions.\n4. Avoid repetitive or overly simplistic questions, ensuring diversity and depth.\n5. The questions must be self-contained and should not require the provided text as background to be understood.\nPlease rewrite the following text into related questions, and strictly follow the format below for output:\n{\n    "question

In [12]:
refined_dataset.save_to_disk("path")

Saving the dataset (3/3 shards): 100%|██████████| 210000/210000 [00:01<00:00, 156989.02 examples/s]
